## Fast Variational Bayesian Linear State-Space Model

This notebook is based on the eponymous [paper](https://www.researchgate.net/profile/Jaakko_Luttinen/publication/256121415_Fast_Variational_Bayesian_Linear_State-Space_Model/links/02e7e535a2e9b467ad000000.pdf?origin=publication_list) by [Jaakko_Luttinen](https://www.researchgate.net/profile/Jaakko_Luttinen), the author of [BayesPy](http://bayespy.org/), and on the example in http://bayespy.org/en/latest/examples/lssm.html

### Overview of Linear State Space Models

We consider a class of models characterised by a noisy higher dimensional representation of some lower dimensional 'hidden' state. This can be applied to both static and dynamic (i.e. where ordering is important) datasets.  The basic models are discrete time linear dynamical systems with Gaussian noise. 











In linear state-space models a sequence of $M$-dimensional observations
$\mathbf{Y}=(\mathbf{y}_1,\ldots,\mathbf{y}_N)$ is assumed to be generated
from latent $D$-dimensional states
$\mathbf{X}=(\mathbf{x}_1,\ldots,\mathbf{x}_N)$ which follow a first-order
Markov process:

   \begin{aligned}
   \mathbf{x}_{n} &= \mathbf{A}\mathbf{x}_{n-1} + \mathbf{w}_{\bullet}  
   \\
   \mathbf{y}_{n} &= \mathbf{C}\mathbf{x}_{n} + \mathbf{v}_{\bullet} 
   \end{aligned}

$\mathbf{A}$ is the $D\times D$ state dynamics matrix and $\mathbf{C}$ is the $M\times D$ loading/observation matrix. Usually, the latent space dimensionality $D$ is assumed to be much smaller than the observation space dimensionality $M$ in order to model the dependencies of high-dimensional observations efficiently.

Here the $D$-vector $\mathbf{w}_{\bullet} \thicksim \mathcal{N}(\mathbf{0},\mathbf{Q})$ and $M$-vector $\mathbf{v}_{\bullet} \thicksim \mathcal{N}(\mathbf{0},\mathbf{R})$ are random variables representing the state evolution and observations noises respectively. Both noise sources are uncorrelated from time step to time step and spatially Gaussian distributed with zero mean and covariance matrices $\mathbf{Q}$ and $\mathbf{R}$ respectively.  The noise processes do not have knowledge of the time index; hence we do not use a $t$ subscript for $\mathbf{w}$ or $\mathbf{v}$

There is a useful unifying perspective on linear dynamical systems with Gaussian noise: we can actually consider PCA, Hidden Markov Models, Mixture of Gaussians and Kalman filtering to be limiting cases (see *"A Unifying Review of Linear Gaussian Models"*, S. Roweis, Z. Ghahramani, Neural Computation **11**, 305–345 (1999) )

We can think intuitively about this as follows. At each time step the hidden noise term $\mathbf{w}_{\bullet}$ randomly generates a spherical ball of density (described by $\mathbf{Q}$) in $k$-dimensional state space. In the dynamic case, this density flows around from time step to time step in the field described by the eigenvalues and eigenvectors of the matrix $\mathbf{A}$. In the static case, each point in the dataset is generated independently and identically so the state dynamics matrix $\mathbf{A}=\mathbf{0}$ and the ball remains centred on the origin. 

In each case the ball is then stretched and rotated into a $p$-dimensional observation space by the matrix $\mathbf{C}$, where it looks like a $k$-dimensional pancake. This pancake is then convolved with the observation noise to get the final convariance model for $\mathbf{y}$.


|             | Continuous States | Discrete States     |
|------------ | -------------     | ---------------     |
|**Dynamic**  | Kalman filter     | Hidden Markov Model |
| **Static**  | Probabilistic PCA, SPCA, PCA, Factor Analysis$^{\dagger}$ |  Gaussian Mixture  | 
$\dagger$ *depending on choice of observation noise covariance matrix* $\mathbf{R}$. 



### Linear State Space Models with BayesPy

In order to construct the model in [BayesPy](http://bayespy.org/), first import relevant nodes:

In [ ]:
import numpy as np
np.random.seed(1)
%matplotlib inline

from bayespy.nodes import GaussianARD, GaussianMarkovChain, Gamma, Dot

In [ ]:
M = 30

There will be 400 data vectors:

In [ ]:
N = 400

Let us use 10-dimensional latent space:

In [ ]:
D = 10

The state dynamics matrix $\mathbf{A}$ has ARD prior:

In [ ]:
alpha = Gamma(1e-5,
               1e-5,
               plates=(D,),
               name='alpha')
A = GaussianARD(0,
                 alpha,
                 shape=(D,),
                 plates=(D,),
                 name='A')

Note that $\mathbf{A}$ is a $D\times{}D$-dimensional matrix.
However, in BayesPy it is modelled as a collection (``plates=(D,)``) of
$D$-dimensional vectors (``shape=(D,)``) because this is how the variables
factorize in the posterior approximation of the state dynamics matrix in
``GaussianMarkovChain``.  The latent states are constructed as

In [ ]:
X = GaussianMarkovChain(np.zeros(D),
                         1e-3*np.identity(D),
                         A,
                         np.ones(D),
                         n=N,
                         name='X')

where the first two arguments are the mean and precision matrix of the initial
state, the third argument is the state dynamics matrix and the fourth argument
is the diagonal elements of the precision matrix of the innovation noise.  The
node also needs the length of the chain given as the keyword argument ``n=N``.
Thus, the shape of this node is ``(N,D)``.

The linear mapping from the latent space to the observation space is modelled
with the loading matrix which has ARD prior:

In [ ]:
gamma = Gamma(1e-5,
               1e-5,
               plates=(D,),
               name='gamma')
C = GaussianARD(0,
                 gamma,
                 shape=(D,),
                 plates=(M,1),
                 name='C')

Note that the plates for ``C`` are ``(M,1)``, thus the full shape of the node is
``(M,1,D)``.  The unit plate axis is added so that ``C`` broadcasts with ``X``
when computing the dot product:

In [ ]:
F = Dot(C, X, name='F')

This dot product is computed over the $D$-dimensional latent space, thus
the result is a $M\times{}N$-dimensional matrix which is now represented
with plates ``(M,N)`` in BayesPy:

In [ ]:
F.plates

We also need to use random initialization either for ``C`` or ``X`` in order to
find non-zero latent space because by default both ``C`` and ``X`` are
initialized to zero because of their prior distributions.  We use random
initialization for ``C`` and then we must update ``X`` the first time before
updating ``C``:


In [ ]:
C.initialize_from_random()

The precision of the observation noise is given gamma prior:

In [ ]:
tau = Gamma(1e-5,
             1e-5,
             name='tau')

The observations are noisy versions of the dot products:

In [ ]:
Y = GaussianARD(F,tau,name='Y')

The variational Bayesian inference engine is then constructed as:

In [ ]:
from bayespy.inference import VB
Q = VB(X, C, gamma, A, alpha, tau, Y)

Note that ``X`` is given before ``C``, thus ``X`` is updated before ``C`` by
default.

## Data


Now, let us generate some toy data for our model.  Our true latent space is four
dimensional with two noisy oscillator components, one random walk component and
one white noise component.

In [ ]:
w = 0.3
a = np.array([[np.cos(w), -np.sin(w), 0, 0], 
               [np.sin(w), np.cos(w),  0, 0], 
               [0,         0,          1, 0],
               [0,         0,          0, 0]])

The true linear mapping is just random, i.e. we sample $\mathbf{C}$ entries from a Gaussian:

In [ ]:
c = np.random.randn(M,4)

Then, generate the latent states and the observations using the model equations:

In [ ]:
x = np.empty((N,4))
f = np.empty((M,N))
y = np.empty((M,N))
x[0] = 10*np.random.randn(4)
f[:,0] = np.dot(c,x[0])
y[:,0] = f[:,0] + 3*np.random.randn(M)
for n in range(N-1):
     x[n+1] = np.dot(a,x[n]) + [1,1,10,10]*np.random.randn(4)
     f[:,n+1] = np.dot(c,x[n+1])
     y[:,n+1] = f[:,n+1] + 3*np.random.randn(M)

We want to simulate missing values, thus we create a mask which randomly removes
80% of the data:

In [ ]:
from bayespy.utils import random
mask = random.mask(M, N, p=0.2)
Y.observe(y, mask=mask)

## Inference

As we did not define plotters for our nodes when creating the model, it is done
now for some of the nodes:

In [ ]:
import bayespy.plot as bpplt
X.set_plotter(bpplt.FunctionPlotter(center=True, axis=-2))
A.set_plotter(bpplt.HintonPlotter())
C.set_plotter(bpplt.HintonPlotter())
tau.set_plotter(bpplt.PDFPlotter(np.linspace(0.02, 0.5, num=1000)))

This enables plotting of the approximate posterior distributions during VB
learning.  The inference engine can be run using ``VB.update()`` method:

In [ ]:
Q.update(repeat=10)

The iteration progresses a bit slowly, thus we'll consider parameter expansion
to speed it up.

### Parameter expansion

Section [Parameter Expansion](http://bayespy.org/en/latest/user_guide/inference.html#sec-parameter-expansion) discusses parameter expansion for state-space models to speed up inference.  It is based on a rotating the latent
space such that the posterior in the observation space is not affected:


$\mathbf{y}_n = \mathbf{C}\mathbf{x}_n = (\mathbf{C}\mathbf{R}^{-1}) (\mathbf{R}\mathbf{x}_n) \,.$

Thus, the transformation is
$\mathbf{C}\rightarrow\mathbf{C}\mathbf{R}^{-1}$ and
$\mathbf{X}\rightarrow\mathbf{R}\mathbf{X}$.  In order to keep the
dynamics of the latent states unaffected by the transformation, the state
dynamics matrix $\mathbf{A}$ must be transformed accordingly:



$\mathbf{R}\mathbf{x}_n = \mathbf{R}\mathbf{A}\mathbf{R}^{-1}
\mathbf{R}\mathbf{x}_{n-1} \,,$

resulting in a transformation $\mathbf{A}\rightarrow\mathbf{R}\mathbf{A}\mathbf{R}^{-1}$.  For more
details, refer to [Luttinen 2013](http://bayespy.org/en/latest/references.html#luttinen-2013) and [Luttinen 2010](http://bayespy.org/en/latest/references.html#luttinen-2010).  In BayesPy,
the transformations are available in ``bayespy.inference.vmp.transformations``:

In [ ]:
from bayespy.inference.vmp import transformations

The rotation of the loading matrix along with the ARD parameters is defined as:

In [ ]:
rotC = transformations.RotateGaussianARD(C, gamma)

For rotating ``X``, we first need to define the rotation of the state dynamics
matrix:

In [ ]:
rotA = transformations.RotateGaussianARD(A, alpha)

Now we can define the rotation of the latent states:

In [ ]:
rotX = transformations.RotateGaussianMarkovChain(X, rotA)

The optimal rotation for all these variables is found using rotation optimizer:

In [ ]:
R = transformations.RotationOptimizer(rotX, rotC, D)

Set the parameter expansion to be applied after each iteration:

In [ ]:
Q.callback = R.rotate

Now, run iterations until convergence:

In [ ]:
Q.update(repeat=1000)

## Results

Because we have set the plotters, we can plot those nodes as:

In [ ]:
Q.plot(X, A, C, tau)
bpplt.pyplot.show()

There are clearly four effective components in ``X``: random walk (component
number 1), random oscillation (6 and 10), and white noise (9).  These dynamics
are also visible in the state dynamics matrix Hinton diagram.  Note that the
white noise component does not have any dynamics.  Also ``C`` shows only four
effective components.  The posterior of ``tau`` captures the true value
$3^{-2}\approx0.111$ accurately.  We can also plot predictions in the
observation space:

In [ ]:
bpplt.plot(F, center=True)
bpplt.pyplot.show()

We can also measure the performance numerically by computing root-mean-square
error (RMSE) of the missing values:

In [ ]:
from bayespy.utils import misc
misc.rmse(y[~mask], F.get_moments()[0][~mask])

This is relatively close to the standard deviation of the noise (3), so the
predictions are quite good considering that only 20% of the data was used.